In [2]:
import pandas as pd
import numpy as np

np.random.seed(42)

# 1. 기본 feature 생성
df = pd.DataFrame({
    'seat': np.random.randint(300, 2000, size=100),
    'marketing': np.random.randint(1000, 5000, size=100),
    'star_power': np.random.randint(1, 6, size=100)
})

# 2. target 컬럼 계산해서 따로 추가
df['audience'] = (
    df['seat'] * 10 +
    df['marketing'] * 1.5 +
    df['star_power'] * 800 +
    np.random.normal(0, 1000, size=100)
)


In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

X = df[['seat', 'marketing', 'star_power']]
y = df['audience']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)



RandomForestRegressor(random_state=42)

In [4]:
def predict_audience_rf(seat, marketing, star_power):
    input_data = pd.DataFrame([{
        'seat': seat,
        'marketing': marketing,
        'star_power': star_power
    }])
    pred = rf_model.predict(input_data)[0]
    return int(pred)


In [5]:
seat = 1000
marketing = 2000
star_power = 5

predicted = predict_audience_rf(seat, marketing, star_power)
print(f"🎯 랜덤포레스트 예측 관객 수: {predicted:,}명")

🎯 랜덤포레스트 예측 관객 수: 17,623명


In [11]:
import os
import json
import requests
import time
import pandas as pd
import numpy as np
#from sklearn.ensemble import RandomForestRegressor  # 실제 모델 대신 고정값 사용 (여기서는 데모용)
from dotenv import load_dotenv
from openai import AzureOpenAI

# 환경변수 로드
load_dotenv()

# ============================================
# 1. 유틸리티 함수: 대화 로그 프린트 및 GPT 프롬프트 생성
# ============================================
def print_pretty_messages(messages):
    print("\n📨 메시지 대화 로그\n" + "-" * 40)
    for message in reversed(messages.data):  # 최신순으로 출력
        role = "🧑 사용자" if message.role == "user" else "🤖 GPT"
        print(f"\n[{role}]")
        for block in message.content:
            if block.type == "text":
                print(block.text.value)
        print("-" * 40)

def make_prediction_prompt(genre, seat, region, marketing, star_power, audience_pred, weekday_rate, weekend_rate):
    return f"""
감사합니다. 과거 유사 {genre} 데이터를 참고하고, 

Random Forest Regressor 모델을 적용해 예측해보겠습니다.

잠시만 기다려주세요…

▶ 장르: {genre}
▶ 지역: {region}
▶ 좌석 수: {seat:,}석
▶ 마케팅 예산: {marketing:,}만 원
▶ 스타 파워: {star_power}점

이에 기반한 예측 모델(Random Forest Regressor)을 적용한 결과:

▶ 예상 관객 수는 약 {audience_pred:,}명 (±500명) 정도로 추정됩니다.
▶ 평일 평균 점유율은 {weekday_rate}%,
   주말 평균 점유율은 {weekend_rate}%로 예상됩니다.

(대시보드 링크 안내)
대시보드에서 일자별 누적 관객 추이 그래프와 유사 공연 대비 비교 바차트를 보실 수 있습니다.

다른 정보도 궁금하신가요? 예를 들어 손익분기점, ROI 분석도 가능합니다.
"""

# ============================================
# 2. 예측 함수 (데모용 고정값 반환)
# ============================================
def predict_audience_rf(seat, marketing, star_power):
    # 데모용: 항상 8,500명, 평일 60%, 주말 85% 반환
    return 8500, 60, 85

# ============================================
# 3. CLU 호출: Azure CLU API에서 사용자 발화를 분석
# ============================================
def call_clu_api(text):
    clu_endpoint_base = os.getenv("clu_endpoint")  # 예: https://language5team.cognitiveservices.azure.com
    clu_api_key = os.getenv("clu_key")
    clu_id = os.getenv("clu_id")
    project_name = "PPCLU"
    deployment_name = "ppv1-test"
    headers = {
        "Ocp-Apim-Subscription-Key": clu_api_key,
        "Content-Type": "application/json",
        "Apim-Request-Id": clu_id
    }
    # 최신 버전 URL (전체 경로 포함)
    url = f"{clu_endpoint_base}/language/analyze-conversations?api-version=2024-11-15-preview"
    body = {
        "kind": "Conversation",
        "analysisInput": {
            "conversationItem": {
                "id": "1",
                "participantId": "user",
                "text": text,
                "modality": "text",
                "language": "ko"
            }
        },
        "parameters": {
            "projectName": project_name,
            "deploymentName": deployment_name,
            "verbose": True,
            "stringIndexType": "TextElement_V8"
        }
    }
    response = requests.post(url, headers=headers, json=body)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"❌ CLU API 호출 실패: {response.status_code}")
        print(response.text)
        return None

# (rag_agent_utils.py에 정의된 함수들을 import)
from rag_agent_utils import parse_clu_response, build_kopis_query, make_gpt_prompt

# ============================================
# 4. Azure OpenAI GPT 및 스레드 설정 (RAG)
# ============================================
client = AzureOpenAI(
    azure_endpoint=os.getenv("oai_assistant_endpoint"),
    api_key=os.getenv("oai_assistant_key"),
    api_version="2024-05-01-preview"
)

assistant = client.beta.assistants.create(
    model="gpt-4o",  # 모델 배포 이름 (예: gpt-4o)
    instructions="""너는 한국의 공연 정보를 잘 알고 있는 문화예술 전문가 도우미야.
사용자가 필요한 정보를 단계별로 물어보고, 모든 정보를 수집한 후 최종 예측 결과와 분석 정보를 제공해줘.
응답은 정중하고 친절한 말투로, 사용자가 추가 질문을 할 수 있도록 유도해줘.""",
    tools=[{"type": "file_search"}, {"type": "code_interpreter"}],
    tool_resources={"file_search": {"vector_store_ids": ["vs_rk9EOjktnff6RYK12Hx5jn7J"]}, "code_interpreter": {"file_ids": []}},
    temperature=0.3,
    top_p=0.95
)

# ============================================
# 5. GPT 호출 함수 (최종 프롬프트 전송)
# ============================================
def request_gpt(prompt):
    azure_oai_endpoint = os.getenv("oai_endpoint")  # 예: 전체 URL
    azure_oai_key = os.getenv("oai_key")
    headers = {
        "Content-Type": "application/json",
        "api-key": azure_oai_key
    }
    body = {
        "messages": [
            {
                "role": "system",
                "content": "공연 산업에 대한 전문 지식을 활용하여, 단계별로 추가 정보를 질문하고 최종 분석 결과를 제공해 주세요."
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 1000
    }
    response = requests.post(azure_oai_endpoint, headers=headers, json=body)
    if response.status_code == 200:
        response_json = response.json()
        content_text = response_json['choices'][0]['message']['content']
        citations = response_json['choices'][0]['message'].get('context', {}).get('citations', [])
        return content_text, citations
    else:
        return response.status_code, response.text

# ============================================
# 6. 인터랙티브 대화 흐름: 초기 질문부터 최종 응답까지
# ============================================
def interactive_conversation():
    # ① 초기 사용자 질문 (관객 예측 요청)
    initial_query = "이번에 새로 준비 중인 뮤지컬의 관객 수를 예측하고 싶어요."
    print("사용자 질문:", initial_query)
    
    # ② 챗봇의 추가 정보 요청 (Assistant가 질문 메시지 생성)
    assistant_initial = ("안녕하세요! 새 공연의 관객 수 예측을 위해 몇 가지 정보가 필요합니다.\n"
                         "공연 장르, 공연장 규모(좌석 수), 지역, 그리고 대략적인 마케팅 예산 및 출연진(스타 파워)을 알려주세요.")
    print("\n[챗봇]:", assistant_initial)
    
    # ③ 사용자로부터 추가 정보 입력 받기 (여기서는 한 번에 입력)
    user_response = input("\n[사용자 입력] (예: 뮤지컬, 1000, 서울, 2000, 5): ")
    # 입력 형식: "뮤지컬, 1000, 서울, 2000, 5"
    try:
        genre, seat_str, region, marketing_str, star_power_str = [s.strip() for s in user_response.split(",")]
        seat = int(seat_str)
        marketing = int(marketing_str)
        star_power = int(star_power_str)
    except Exception as e:
        print("입력 형식 오류. 예: 뮤지컬, 1000, 서울, 2000, 5")
        return
    
    # ④ 예측 수행 (랜덤포레스트 데모 함수 사용)
    audience_pred, weekday_rate, weekend_rate = predict_audience_rf(seat, marketing, star_power)
    print(f"\n[예측] 예상 관객 수: {audience_pred:,}명, 평일: {weekday_rate}%, 주말: {weekend_rate}%")
    
    # ⑤ CLU API 호출로 추출된 엔터티 (여기서는 간단하게 고정값 사용; 실제로는 CLU 결과 활용)
    # 만약 CLU가 이미 호출되어서 {"장르": "뮤지컬", "지역": "서울", "일정": "5월"} 등이 추출되었다고 가정
    extracted_entities = {"장르": genre, "지역": region, "일정": "5월"}
    print("\n[CLU 추출 엔터티]:", extracted_entities)
    
    # ⑥ KOPIS 검색 조건 생성 (엔터티 기반)
    kopis_query = build_kopis_query(extracted_entities)
    print("\n[KOPIS 검색 조건]:", kopis_query)
    
    # ⑦ (더미) KOPIS API 호출: 더미 공연 결과 반환
    def call_kopis_api(kopis_query):
        return (f"- 더미 공연 결과: {kopis_query.get('genre', '공연')} 공연 in {kopis_query.get('region', '서울')} "
                f"from {kopis_query.get('startDate', '날짜')} to {kopis_query.get('endDate', '날짜')}.")
    kopis_results = call_kopis_api(kopis_query)
    print("\n[KOPIS 검색 결과]:", kopis_results)
    
    # ⑧ 최종 GPT 프롬프트 생성 (예측 결과, KOPIS 결과 포함)
    final_prompt = make_prediction_prompt(
        genre=genre,
        seat=seat,
        region=region,
        marketing=marketing,
        star_power=star_power,
        audience_pred=audience_pred,
        weekday_rate=weekday_rate,
        weekend_rate=weekend_rate
    )
    print("\n[최종 GPT 프롬프트]:\n", final_prompt)
    
    # ⑨ GPT 호출하여 최종 응답 생성
    gpt_response, citations = request_gpt(final_prompt)
    print("\n🧠 GPT 최종 응답:\n", gpt_response)
    print("\n📚 출처:\n", citations)
    
    # ⑩ 추가 대화: Azure OpenAI 챗봇 스레드 실행 (실제 대화)
    thread = client.beta.threads.create()
    additional_input = input("\n[추가 대화 입력] (내용을 입력하세요): ")
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=additional_input
    )
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id
    )
    while run.status in ['queued', 'in_progress', 'cancelling']:
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id
        )
    if run.status == 'completed':
        messages = client.beta.threads.messages.list(thread_id=thread.id)
        print_pretty_messages(messages)
    elif run.status == 'requires_action':
        print("추가 작업 필요")
    else:
        print("스레드 상태:", run.status)

if __name__ == "__main__":
    interactive_conversation()


사용자 질문: 이번에 새로 준비 중인 뮤지컬의 관객 수를 예측하고 싶어요.

[챗봇]: 안녕하세요! 새 공연의 관객 수 예측을 위해 몇 가지 정보가 필요합니다.
공연 장르, 공연장 규모(좌석 수), 지역, 그리고 대략적인 마케팅 예산 및 출연진(스타 파워)을 알려주세요.
입력 형식 오류. 예: 뮤지컬, 1000, 서울, 2000, 5
